## loss

### 概念

#### 信息量
对于一个随机事件，其发生的概率越大，包含的信息量越少，信息量计算公式如下：<br>
$$I(x_j) = -log(p(x_j))$$
根据公式可以知道，概率为1的事件发生对应的信息量为0。

#### 熵
熵来表示信息量的期望，计算公式如下：
$$H(X) = \sum_{j} p(x_j)I(x_j) = -\sum_{j} p(x_j)log(p(x_j))$$

#### KL散度（相对熵）
对于同一个随机变量X，如果其真实的分别为$P(X)$，预测分布为$Q(X)$，则可以使用KL散度（Kullback-Leibler divergence）来衡量这两个分布的差异。
例如在深度学习中有一个10分类（每个类别都是互斥的）的问题，当前样本label为5，其对应的真实概率分布可以使用one-hot编码来表示为$[0,0,0,0,1,0,0,0,0,0]$，如果模型的预测分布为$[0.1,0.1,0.1,0.2,0.3,0.1,0.1,0.0,0.0,0.0]$，那么KL散度的计算公式如下：
$$D_{KL}(p|q)=\sum_{j} p(x_j)log(\frac{p(x_j)}{q(x_j)})$$
- KL散度越小，表示两个分布越接近
- 根据公式可以知道，只需要考虑正确类别对应的预测概率即可
$$D = log(1/0.3)$$

#### 交叉熵
我们将KL散度公式进一步推导：
$$D_{KL}(p|q)=\sum_{j} p(x_j)log(p(x_j) - \sum_{j}p(x_i)log(q(x_j))\\
= -H(p(x)) - \sum_{j}p(x_j)log(q(x_j))$$
KL散度的前半部分就是熵，由于label固定不变故熵也是不变的，那么在判定label与predicts“距离”的时候只需要关注后半部分即可，即交叉熵！在深度学习中，交叉熵可以作为loss，评估模型。
$$H(p,q) = -\sum_{j}p(x_j)log(q(x_j))$$
仍然以上面为例：<br>
当前样本label为5，其对应的真实概率分布可以使用one-hot编码来表示为$[0,0,0,0,1,0,0,0,0,0]$，如果模型的预测分布为$[0.1,0.1,0.1,0.2,0.3,0.1,0.1,0.0,0.0,0.0]$
那么$$H=-log(0.3)$$

#### softmax
前面提到了多次预测概率的问题，那么在深度学习中如何得到预测概率呢？使用softmax层就可以获取概率。<br>
$$p_j = \frac{exp(s_j)}{\sum_{j=0} exp(s_j)}$$
例如:对于一个样本分类（为了方便计算，类别数量=3），经过前面网络层的处理得到了$[2.8,-0.2,4.6]$，经过softmax处理后得到的概率表示为$[0.14,0.007,0.85]$

### BCE
binary cross entropy就是二分类的交叉熵，为了直观理解，举一个2分类的例子。假如当前batch包含了2个样本，label使用one-hot编码表示，即y=[[1,0],[0,1]]，模型的预测输出为p=[[0.8,0.2],[0.4,0.6]]，则对应的损失就是：
$$L = (-log(0.8) - log(0.6))/2$$
<br>
BCE的计算方法与使用逻辑回归代价损失函数计算交叉熵得到的结果一模一样，逻辑回归的损失函数为：
$$J(w)=-\frac{1}{m}\sum_{j=1}^{m}[y_j log(h_w(x_j)) + (1-y_j)log(1-h_w(x_j))]$$
例如：对于2分类问题，batch包含2个样本，label不用one-hot编码表示，即y=[0,1]，预测p=[0.2,0.6]，那么损失为：
$$L = [-0*log(0.2) - (1-0)*log(1-0.2)] + [-1*log(0.6) - (1-1)*log(1-0.6)] \\
   = [-log(0.8) - log(0.6)]/2$$

#### nn.BCELoss()
BCELoss的实例对象接收的参数为：
- input: (N,*,), N代表样本数，成员取值范围在[0.0,1.0]之内，代表样本为正(=1)的概率值
- target: (N,*,)，N代表样本数，与input的shape是一样的，成员都是0.0或1.0


In [15]:
import torch
import torch.nn as nn
m = nn.Sigmoid()
loss = nn.BCELoss()

#### Demo1


In [20]:
x1 = torch.randn(5,requires_grad=True)
y1 = m(x1)
print(y1)
target = torch.empty(5).random_(2)
print(target)
output = loss(y1,target)
print(output)

tensor([0.7871, 0.2998, 0.6116, 0.6319, 0.3875], grad_fn=<SigmoidBackward>)
tensor([0., 1., 1., 0., 1.])
tensor(1.0381, grad_fn=<BinaryCrossEntropyBackward>)


使用数学公式计算

In [22]:
output = -torch.sum(target*torch.log(y1) + (1-target)*torch.log(1-y1))/y1.size(0)
output

tensor(1.0381, grad_fn=<DivBackward0>)

#### Demo2

In [23]:
x2 = torch.randn(5,3,requires_grad=True)
print(x2)
y2 = m(x2)
print(y2)
#注意，这里的每一行也是
target = torch.tensor([[0,1,0],
                       [0,0,1],
                       [0,1,0],
                       [0,1,0],
                       [1,0,0]],dtype=torch.float)
output = loss(y2,target)
print(output)

tensor([[ 1.5614, -0.1124, -0.2877],
        [-0.1296, -0.2690, -1.9163],
        [ 0.7400, -0.7023, -0.1783],
        [-1.0932, -0.8159, -0.8373],
        [-1.1140,  1.5444, -0.7263]], requires_grad=True)
tensor([[0.8265, 0.4719, 0.4286],
        [0.4676, 0.4332, 0.1283],
        [0.6770, 0.3313, 0.4555],
        [0.2510, 0.3066, 0.3021],
        [0.2471, 0.8241, 0.3260]], grad_fn=<SigmoidBackward>)
tensor(0.9679, grad_fn=<BinaryCrossEntropyBackward>)


In [30]:
output = -torch.sum(target*torch.log(y2) + (1-target)*torch.log(1-y2)) / (
                            y2.size(0)*y2.size(1))
output

tensor(0.9679, grad_fn=<DivBackward0>)